使用卷积层实现平均池化

In [2]:
import torch
from torch import nn

def avg_pool_using_conv(input_tensor, kernel_size, stride):
    if stride is None:
        stride = kernel_size

    # 获取输入通道数 (batch, channel, height, weight)
    in_channels = input_tensor.shape[1]

    # 标准卷积会默认将输入的所有通道数据加权汇总
    # 池化针对各个通道进行，池化前后通道数不变
    conv_layer = nn.Conv2d(in_channels=in_channels,
                           out_channels=in_channels,
                           kernel_size=kernel_size,
                           stride=stride,
                           groups=in_channels,
                           bias=False)

    # 初始化权重，平均池化本质上是固定权重卷积
    window_area = kernel_size ** 2
    weight_value = 1.0 / window_area

    # 模拟固定的池化操作，不需要梯度计算
    conv_layer.weight.data.fill_(weight_value)
    conv_layer.weight.requires_grad = False

    return conv_layer(input_tensor)

验证

In [3]:
# 创建一个模拟输入: 1个样本, 2个通道, 4x4大小
X = torch.tensor([
    [
        [0.0, 1.0, 2.0, 3.0],
        [4.0, 5.0, 6.0, 7.0],
        [8.0, 9.0, 10.0, 11.0],
        [12.0, 13.0, 14.0, 15.0]
    ],
    [
        [1.0, 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, 1.0]
    ]
]).unsqueeze(0) # 增加 Batch 维度 -> (1, 2, 4, 4)

print(f"输入形状: {X.shape}")

official_pool = nn.AvgPool2d(kernel_size=2, stride=2)
out_official = official_pool(X)

out_conv = avg_pool_using_conv(X, kernel_size=2, stride=2)

print("\n官方 AvgPool2d 输出:")
print(out_official)

print("\n卷积实现的 AvgPool 输出:")
print(out_conv)

# 使用 allclose 防止浮点数精度误差
is_same = torch.allclose(out_official, out_conv)
print(f"\n两者结果是否一致: {is_same}")

输入形状: torch.Size([1, 2, 4, 4])

官方 AvgPool2d 输出:
tensor([[[[ 2.5000,  4.5000],
          [10.5000, 12.5000]],

         [[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]]]])

卷积实现的 AvgPool 输出:
tensor([[[[ 2.5000,  4.5000],
          [10.5000, 12.5000]],

         [[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]]]])

两者结果是否一致: True
